# 基于TF-IDF算法的关键词 抽取

In [53]:
import jieba.analyse as analyse
import pandas as pd

jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())

- sentence 为待提取文本
- topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
- withWeight为是否一并返回关键字权重，默认为False
- allowPOS仅返回指定词性的词，默认值为空，即不筛选

In [54]:
df = pd.read_csv("./data/technology_news.csv", encoding="utf-8")
df = df.dropna()
df[:5]

,Unnamed: 0,content
0,0,",中新网,1月7日电 恰逢CES 2017拉开大幕，却惊闻“AlphaGo升级版”的M..."
1,1,徐立，商汤科技CEO在谈起本次参展时谈到：“作为一个成立刚刚两年的创业公司，这次参展，一...
2,2,正如最后挑战Master的古力在落败后发表的观点：“人类与人工智能共同探索围棋世界的大幕...
3,3,", SenseFace人脸布控的“黑科技”"
4,4,本届大展最大的看点，无疑是“被誉为2016全美科技第一神股”英伟达的首次CES主题演讲。...


In [55]:
lines = df.content.values.tolist()
content = "".join(lines)
print(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=()))

['用户', '2016', '互联网', '手机', '平台', '人工智能', '百度', '2017', '智能', '技术', '数据', '360', '服务', '直播', '产品', '企业', '安全', '视频', '移动', '应用', '网络', '行业', '游戏', '机器人', '电商', '内容', '中国', '领域', '通过', '发展']


In [56]:
df = pd.read_csv("./data/military_news.csv", encoding="utf-8")
df = df.dropna()
df[:5]

,Unnamed: 0,content
0,0,新华社北京1月8日电(记者董瑞丰)军民融合发展已经上升为国家战略，但现阶段国防工业企业军...
1,1,中国国防工业企业军民融合产业联盟8日在北京宣布成立。刘东奎说，适时成立联盟，坚持“军民融...
2,2,数据显示，我国目前有十几万家民营高新技术企业，很多高新技术企业在新能源、新材料、电子信息...
3,3,对于现阶段军民融合面临的问题，刘东奎列举：从企业外部来看，军民融合管理体制不够顺畅，军民...
4,4,“国防科技工业是军民融合的天然载体和核心领域。”刘东奎说，未来要着力搭建军民协同创新展示...


In [57]:
lines = df.content.values.tolist()
content = "".join(lines)
" ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=()))

'航母 训练 海军 中国 官兵 部队 编队 作战 10 任务 美国 导弹 能力 20 2016 军事 无人机 装备 进行 记者 我们 军队 安全 保障 12 战略 军人 日本 南海 战机'

# 基于TextRank算法的关键词抽取

- jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=("ns", "n", "vn", "v"))
- jieba.analyse.TextRank()

基本思想：

- 将抽取关键词的文本进行分词
- 以固定窗口大小（默认为5，通过span属性调整），根据词之间的共现关系构建图
- 计算图中节点的PageRank，注意无想带权图

In [58]:
df = pd.read_csv("./data/military_news.csv", encoding="utf-8")
df = df.dropna()
lines = df.content.values.tolist()
content = "".join(lines)
" ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=("ns", "n", "vn", "v")))

'中国 海军 训练 美国 部队 进行 官兵 航母 作战 任务 能力 军事 发展 工作 国家 问题 建设 导弹 编队 记者'

In [59]:
" ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=("ns", "n")))

'中国 海军 美国 部队 官兵 航母 军事 国家 任务 能力 导弹 技术 问题 日本 军队 编队 装备 系统 记者 战略'

# LDA主题模型

In [60]:
from gensim import corpora, models, similarities
import gensim

## 载入停用词

In [61]:
stopwords = pd.read_csv("./data/stopwords.txt", index_col=False, quoting=3, sep="\t", names={"stopword"}, encoding="utf-8")
stopwords = stopwords["stopword"].values

In [62]:
df = pd.read_csv("./data/technology_news.csv", encoding="utf-8")
df = df.dropna()
lines = df.content.values.tolist()

In [63]:
sentences = []
for line in lines:
    try:
        segs = jieba.lcut(line)
        segs = filter(lambda x: len(x) > 1, segs)
        segs = filter(lambda x: x not in stopwords, segs)
        sentences.append(segs)
    except Exception as e:
        print(line)
        continue

In [64]:
for word in sentences[5]:
    print(word)

本次
商汤
带来
黄仁勋
展示
遥相呼应
SenseFace
人脸
布控
系统
千万级
人员
库中
300ms
识别
瞬间
锁定目标
功耗
十几
当属
人脸
布控
一大
科技


## 词袋模型

In [72]:
import jieba
import pandas as pd
df = pd.read_csv("./data/technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()

sentences=[]
for line in lines:
    try:
        segs=jieba.lcut(line)
        segs = filter(lambda x:len(x)>1, segs)
        segs = filter(lambda x:x not in stopwords, segs)
        sentences.append(segs)
    except Exception as e:
        print(line)
        continue

In [73]:
# dictionary = corpora.Dictionary(sentences)
# corpus = [dictionary.doc2bow(sentence) for sentence in sentences]
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]
corpus[:5]

[[], [], [], [], []]

In [66]:
lda = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

C:\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:606: RuntimeWarning: invalid value encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [67]:
print(lda.print_topic(3, topn=5))

0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23"


打印出所有主题

In [68]:
for topic in lda.print_topics(num_topics=20, num_words=8):
    print(topic[1])

0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23" + 0.000*"4050.8" + 0.000*"亏损企业" + 0.000*"15.03"
0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23" + 0.000*"4050.8" + 0.000*"亏损企业" + 0.000*"15.03"
0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23" + 0.000*"4050.8" + 0.000*"亏损企业" + 0.000*"15.03"
0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23" + 0.000*"4050.8" + 0.000*"亏损企业" + 0.000*"15.03"
0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23" + 0.000*"4050.8" + 0.000*"亏损企业" + 0.000*"15.03"
0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23" + 0.000*"4050.8" + 0.000*"亏损企业" + 0.000*"15.03"
0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23" + 0.000*"4050.8" + 0.000*"亏损企业" + 0.000*"15.03"
0.000*"176.25%" + 0.000*"52.48" + 0.000*"5.84%" + 0.000*"4971.56" + 0.000*"3.23" + 0.000*"4050.8"

In [69]:
# lda.get_document_topics()